In [1]:
import pandas as pd
from pyArango.connection import *
from pyArango.collection import *

In [2]:
conn = Connection(username="root", password="ngph")

In [3]:
df = pd.read_csv('transfer_network_201812.csv')

In [4]:
db = conn["m2"]

In [5]:
transfer_df = df.groupby(['sender', 'receiver']).agg({
    'transID': 'count',
    'amount': 'sum'
})
transfer_df.columns = ['num_of_trans', 'total_amount']
transfer_df[:10]

,,num_of_trans,total_amount
sender,receiver,,
00055b984cdfbe6b799d5debf8323c64,43026332e53541a19627d074c1833200,1,1500
0006544b05ceb936b9ea7b2af41e8f22,060c23c14d40d3e85d6054783db38a02,1,20000
000708e22b25c950d014c17e0424cbfb,042c9ebffdff74530d48a92d3a6b9ea3,1,500000
00085550d7aa4e4f322a79effc6cc3fc,a6238e443bc03bfc46b3f6799c16ae2c,2,1033143
000c1d6349b283a51bc071e16a8ecb7a,9b3b61b0e9f6a17ad23750e4a6ad24a2,1,300000
000d69f7c9ef2652b631062b6376c606,c23a13b8204936d5a0e581ca7c5c9307,1,227000
000ea5580bdc773a6f7f3f77c8f11f17,c07123020748b6342ad5cafdc06662ab,1,25000
000ec49f2c8a4f9882db8167f6cc4542,b766a2b2775712111f73c354b258e1a2,1,50000
0010058c779f4507618e1a9d341e6929,1c54a9159553e1f321a9fa2b7f7e72b8,2,201000


In [6]:
transfer_df.reset_index(inplace=True)
transfer_df[:10]

,sender,receiver,num_of_trans,total_amount
0,00055b984cdfbe6b799d5debf8323c64,43026332e53541a19627d074c1833200,1,1500
1,0006544b05ceb936b9ea7b2af41e8f22,060c23c14d40d3e85d6054783db38a02,1,20000
2,000708e22b25c950d014c17e0424cbfb,042c9ebffdff74530d48a92d3a6b9ea3,1,500000
3,00085550d7aa4e4f322a79effc6cc3fc,a6238e443bc03bfc46b3f6799c16ae2c,2,1033143
4,000c1d6349b283a51bc071e16a8ecb7a,9b3b61b0e9f6a17ad23750e4a6ad24a2,1,300000
5,000d69f7c9ef2652b631062b6376c606,c23a13b8204936d5a0e581ca7c5c9307,1,227000
6,000ea5580bdc773a6f7f3f77c8f11f17,c07123020748b6342ad5cafdc06662ab,1,25000
7,000ec49f2c8a4f9882db8167f6cc4542,b766a2b2775712111f73c354b258e1a2,1,50000
8,0010058c779f4507618e1a9d341e6929,1c54a9159553e1f321a9fa2b7f7e72b8,2,201000
9,0010058c779f4507618e1a9d341e6929,2606c7cba485fdbcd053fd9f01489ca6,2,163500


In [7]:
users=None
if not db.hasCollection('users'):
    users = db.createCollection(name="users")
else:
    users = db["users"]

In [17]:
all_users = transfer_df['sender'].append(transfer_df['receiver']).unique()
for u in all_users:
    new_u = users.createDocument()
    new_u._key = u
    new_u.save()

===
Unable to establish connection, perhaps arango is not running.
===


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8529): Max retries exceeded with url: /_db/m2/_api/document?collection=users&waitForSync=False (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D0D0E32CD0>: Failed to establish a new connection: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted'))

In [8]:
sendTo = None
if not db.hasCollection(name='sendTo'):
    sendTo = db.createCollection(name='sendTo', className='Edges')
else:
    sendTo = db['sendTo']

In [9]:
for sender, receiver, num_of_trans, total_amount in transfer_df.values:
    new_edge = sendTo.createDocument({
        "_from": "users/" + sender,
        "_to": "users/" + receiver
    })
    new_edge['num_of_trans'] = num_of_trans
    new_edge['total_amount'] = total_amount
    new_edge.save()

===
Unable to establish connection, perhaps arango is not running.
===


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8529): Max retries exceeded with url: /_db/m2/_api/document?collection=sendTo&waitForSync=False (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AE3E3B01C0>: Failed to establish a new connection: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted'))